In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [7]:
import train_and_eval as tae


tae.ex.run(
    named_configs=["configs/gpn/classification_gpn_16.yaml"],
    config_updates={
        "data.dataset": "CoraML",
        # "run.reduced_training_metrics": True,
        # "training.eval_every": 10,
        # "training.stopping_patience": 5,
        # "data.split": "public",
        # "run.num_splits": 1,
        "run.num_splits": 10,
        "run.num_inits": 10,
        "run.log": False,
        "model.model_name": "GPN_LOP",
        "model.sparse_x_prune_threshold": 0.01,
    },
    options={"--force": True},
)

WARNING - train_and_eval - No observers have been added to this run
INFO - train_and_eval - Running command 'run_experiment'
INFO - train_and_eval - Started
INFO - root - Received the following configuration:
INFO - root - RUN
INFO - root - {'experiment_name': 'classification', 'experiment_directory': './saved_experiments', 'reduced_training_metrics': False, 'eval_mode': 'default', 'job': 'train', 'save_model': True, 'gpu': 0, 'num_inits': 10, 'num_splits': 10, 'log': False, 'debug': True, 'ex_type': 'transductive', 'ood_loc': True, 'ood_loc_only': False, 'ood_edge_perturbations': True, 'ood_isolated_perturbations': False}
INFO - root - -----------------------------------------
INFO - root - DATA
INFO - root - {'to_sparse': False, 'split_no': 1, 'dataset': 'CoraML', 'root': './data', 'split': 'random', 'train_samples_per_class': 0.05, 'val_samples_per_class': 0.15, 'test_samples_per_class': 0.8, 'ood_flag': False}
INFO - root - -----------------------------------------
INFO - root - MO

Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=1).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=1).
Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=2).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=2).
Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=3).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=3).
Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=4).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=4).
Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=5).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=5).
Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=None, split=1, init=6).
Completed experiment (model=GPN_LOP, dataset=CoraML, ood_typ

INFO - train_and_eval - Result: {'test_accuracy': 0.8168853288888931, 'test_brier_score': 0.5018387681245804, 'test_ECE': 0.2197151266038418, 'test_confidence_aleatoric_apr': 0.9438039234899016, 'test_confidence_epistemic_apr': 0.9292879989605944, 'test_confidence_structure_apr': nan, 'test_confidence_aleatoric_auroc': 0.8200256223304706, 'test_confidence_epistemic_auroc': 0.7779926073960638, 'test_confidence_structure_auroc': nan, 'test_CE': 0.7492188900709152, 'test_avg_prediction_confidence_aleatoric': 0.5975579965114594, 'test_avg_prediction_confidence_epistemic': 302719.37794921873, 'test_avg_sample_confidence_aleatoric': 0.5975579965114594, 'test_avg_sample_confidence_epistemic': 343275.77640625, 'test_avg_sample_confidence_epistemic_entropy': 32.52051052093506, 'test_avg_sample_confidence_features': 329531.3978515625, 'test_avg_sample_confidence_neighborhood': nan, 'test_average_entropy': 1.1946770095825194, 'val_accuracy': 0.8171529370546341, 'val_brier_score': 0.49879487484693


|                                             |              val |             test |
|:--------------------------------------------|-----------------:|-----------------:|
| accuracy                                    |      0.817153    |      0.816885    |
| brier_score                                 |      0.498795    |      0.501839    |
| ECE                                         |      0.218847    |      0.219715    |
| confidence_aleatoric_apr                    |      0.942861    |      0.943804    |
| confidence_epistemic_apr                    |      0.930374    |      0.929288    |
| confidence_structure_apr                    |    nan           |    nan           |
| confidence_aleatoric_auroc                  |      0.817259    |      0.820026    |
| confidence_epistemic_auroc                  |      0.780804    |      0.777993    |
| confidence_structure_auroc                  |    nan           |    nan           |
| CE                                          |      

In [5]:
from gpn.experiments.multiple_run_experiment import MultipleRunExperiment
from gpn.experiments.transductive_experiment import TransductiveExperiment
from gpn.utils.config import (
    DataConfiguration,
    ModelConfiguration,
    RunConfiguration,
    TrainingConfiguration,
)


def create_experiment(model_name):
    run_cfg = RunConfiguration(
        job="train",
        eval_mode="default",
        experiment_directory="./.cache",
        save_model=True,
        gpu=0,
        experiment_name="ood_loc",
        num_inits=1,
        num_splits=1,
    )
    data_cfg = DataConfiguration(
        dataset="CoraML",
        split_no=1,
        root="./data",
        ood_flag=False,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )

    model_cfg = ModelConfiguration(
        model_name=model_name,
        seed=42,
        init_no=1,
        dim_hidden=64,
        dropout_prob=0.5,
        K=10,
        add_self_loops=True,
        maf_layers=0,
        gaussian_layers=0,
        use_batched_flow=True,
        loss_reduction="sum",
        approximate_reg=True,
        flow_weight_decay=0.0,
        pre_train_mode="flow",
        alpha_evidence_scale="latent-new",
        alpha_teleport=0.1,
        entropy_reg=0.0001,
        dim_latent=16,
        radial_layers=10,
        sparse_propagation=True,
        sparse_x_prune_threshold=0.01
    )

    train_cfg = TrainingConfiguration(
        epochs=100000,
        stopping_mode="default",
        stopping_patience=50,
        stopping_restore_best=True,
        stopping_metric="val_CE",
        stopping_minimize=True,
        finetune_epochs=0,
        warmup_epochs=5,
        lr=0.01,
        weight_decay=0.001,
    )

    return MultipleRunExperiment(run_cfg, data_cfg, model_cfg, train_cfg)


gpn_e = create_experiment("GPN")
lop_e = create_experiment("GPN_LOP")
res_lop = lop_e.run()
res = gpn_e.run()
res_lop

Starting experiment (model=GPN_LOP, dataset=CoraML, ood_type=leave_out_classes, split=1, init=1).
Epoch 1/5:


RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Bool

In [ ]:
import pandas as pd
from gpn.utils.utils import results_dict_to_df

df_lop = results_dict_to_df(res_lop)
df = results_dict_to_df(res)

pd.set_option("display.max_rows", None)
(pd.DataFrame(
    dict(
        val_gpn=df["val"],
        test_gpn=df["test"],
        val_lop=df_lop["val"],
        test_lop=df_lop["test"],
    )
)).round(4).dropna()

In [ ]:
from gpn.data.dataset_manager import DatasetManager
from gpn.data.dataset_provider import InMemoryDatasetProvider
import torch_sparse as ts

ds = InMemoryDatasetProvider(
    DatasetManager(
        dataset="AmazonPhotos",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )
).to_sparse()

batch = list(ds.loader())[0]

batch.adj_t.dtype()

In [ ]:
N = batch.x.shape[0]
adj_t = batch.adj_t.cuda()
adj_t

In [5]:
import torch_sparse as ts

adj_t = ts.SparseTensor.from_dense(
    torch.Tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
).cuda()
x = torch.Tensor([1, 10, 100]).view(-1, 1).cuda()
N = x.shape[0]
adj_t /= x
print(adj_t.dtype())
adj_t.set_diag(torch.tensor([6,2,3], dtype=torch.float32).cuda()).to_dense()

torch.float32


tensor([[6.0000, 1.0000, 0.0000],
        [0.1000, 2.0000, 0.1000],
        [0.0000, 0.0100, 3.0000]], device='cuda:0')

In [19]:
from gpn.layers.appnp_propagation import APPNPPropagation

prop = APPNPPropagation(1, 0, 0, normalization="sym")
x = ts.SparseTensor.eye(N, dtype=torch.float32, device="cuda")  # type: ignore

p: ts.SparseTensor = prop(x, adj_t)

p.to_dense()

tensor([[0.5000, 0.6455, 0.0000],
        [0.0645, 0.8333, 0.0908],
        [0.0000, 0.0091, 0.9901]], device='cuda:0')

In [20]:
prop = APPNPPropagation(1, 0, 0, normalization="sym")
x = torch.diag(torch.ones(N, device="cuda"))

p: ts.SparseTensor = prop(x, adj_t)

p

tensor([[0.5000, 0.6455, 0.0000],
        [0.0645, 0.8333, 0.0908],
        [0.0000, 0.0091, 0.9901]], device='cuda:0')

In [ ]:
ts.SparseTensor.from_edge_index(torch.tensor([[0,1],[0,1]]),sparse_sizes=(10,3)).dtype()

In [43]:
ts.SparseTensor.from_.get_diag()

TypeError: <lambda>() missing 1 required positional argument: 'self'